# Massive Data Processing







## Spark practice

<b>Student: David Sánchez</b>

In [70]:
import pyspark
sc = pyspark.SparkContext('local[*]')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-4bc56731f3c8>:2 

### 1.-Practice: Trending Topics & analysis sentiment (5.0%)


#### Trending Topics

In [2]:
import os, shutil
import json
import unicodedata
if os.path.exists("Results/Tweets_ES.json"): 
    shutil.rmtree("Results/Tweets_ES.json")

input = sc.textFile("Datasets/Tweets/tweets_es.json")
tweets = input.map(lambda x: json.loads(x))
print "Total tweets: ", tweets.count()

tweets_es = tweets.filter(lambda t: "es" in t["lang"])
print "Spanish tweets: ", tweets_es.count()

tweets_es_hashtags = tweets_es.filter(lambda t: t["entities"]["hashtags"] != [] )
print  "Tweets with hashtags: ", tweets_es_hashtags.count()

hashtags = tweets_es_hashtags.flatMap(lambda t: map(lambda h: (unicodedata.normalize('NFKD', h["text"]).encode('ascii','ignore'),1), t["entities"]["hashtags"]))
print "Hashtags: ", hashtags.count()

trending_hashtags = hashtags.reduceByKey(lambda a, b: a + b)
print "Hashtags reduced: ", trending_hashtags.count()
print trending_hashtags.take(10)

Total tweets:  19166
Spanish tweets:  15028
Tweets with hashtags:  3495
Hashtags:  5286
Hashtags reduced:  2947
[('VineDeLaAbuela', 1), ('', 1), ('PaisajeCulturalCafetero', 1), ('RMUCL', 11), ('PorSiNoLoViste', 2), ('industria40', 1), ('Mazatepec', 1), ('wallapop', 1), ('29A', 1), ('SomosServidoresPublicos', 2)]


#### Top N

In [93]:
trending_sorted = trending_hashtags.takeOrdered(10, key=lambda t: -t[1])
print trending_sorted

if os.path.exists("Results/Tweets_ES.json"): 
    shutil.rmtree("Results/Tweets_ES.json")
# trending_sorted.saveAsTextFile('Results/TopN.txt')


[('MTVMiaw', 196), ('Vota2ParaQueSigaPresidente', 117), ('FelizMartes', 84), ('TuitUtil', 65), ('LibertadDePrensa', 64), ('TeCaesTeLevantasFelices27Mica', 51), ('MTVSnapMexDanna', 39), ('MTVPopMBautista', 37), ('MtvIconoMBautista', 34), ('DebateReal', 34)]


#### Sentiment

In [132]:
print tweets_es_hashtags.take(1)

[{u'contributors': None, u'truncated': False, u'text': u'@DisneySpain @TiniStoessel LIBERA LOGOOO \n#TINI', u'is_quote_status': False, u'in_reply_to_status_id': 727492075894099969, u'id': 727497968731959296, u'favorite_count': 0, u'source': u'<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', u'retweeted': False, u'coordinates': None, u'timestamp_ms': u'1462284006661', u'entities': {u'user_mentions': [{u'indices': [0, 12], u'screen_name': u'DisneySpain', u'id': 117748602, u'name': u'Disney Espa\xf1a', u'id_str': u'117748602'}, {u'indices': [13, 26], u'screen_name': u'TiniStoessel', u'id': 550281859, u'name': u'TINI', u'id_str': u'550281859'}], u'symbols': [], u'hashtags': [{u'indices': [42, 47], u'text': u'TINI'}], u'urls': []}, u'in_reply_to_screen_name': u'DisneySpain', u'in_reply_to_user_id': 117748602, u'retweet_count': 0, u'id_str': u'727497968731959296', u'favorited': False, u'user': {u'follow_request_sent': None, u'profile_use_background_image

In [142]:
tweets_temp = tweets_es_hashtags.map(lambda t: (t["text"],t["id"],t["entities"]["hashtags"]))
print tweets_temp.take(1)

[(u'@DisneySpain @TiniStoessel LIBERA LOGOOO \n#TINI', 727497968731959296, [{u'indices': [42, 47], u'text': u'TINI'}])]


In [139]:
# Only tweets text
tweets_text = tweets_es_hashtags.map(lambda t: unicodedata.normalize('NFKD',t["text"]).encode('ascii','ignore').lower())
print tweets_text.take(2)

['@disneyspain @tinistoessel libera logooo \n#tini', 'rt @amctv_es: el grupo esta separado... conseguiran escapar? llegaran a baja? el proximo lunes nuevo episodio! #feartwd\nhttps://t.co/vqw...']


In [104]:
# Number of words
tweets_words = tweets_es_hashtags.map(lambda t: (t["text"], t["text"].split(" "))).mapValues(lambda x: len(x))
print tweets_words.take(2)

[(u'@DisneySpain @TiniStoessel LIBERA LOGOOO \n#TINI', 5), (u'RT @amctv_es: El grupo est\xe1 separado... \xbfconseguir\xe1n escapar? \xbfllegar\xe1n a Baja? \xa1El pr\xf3ximo lunes nuevo episodio! #FearTWD\nhttps://t.co/vQW\u2026', 17)]


In [107]:
# Hashtags
tweets_hashtag = tweets_es_hashtags.map(lambda t: (t["text"], map(lambda h: h["text"], t["entities"]["hashtags"])))\
                    .flatMapValues(lambda x: x)
print tweets_hashtag.take(2)

[(u'@DisneySpain @TiniStoessel LIBERA LOGOOO \n#TINI', u'TINI'), (u'RT @amctv_es: El grupo est\xe1 separado... \xbfconseguir\xe1n escapar? \xbfllegar\xe1n a Baja? \xa1El pr\xf3ximo lunes nuevo episodio! #FearTWD\nhttps://t.co/vQW\u2026', u'FearTWD')]


In [129]:
tweets_words = tweets_hashtag.map(lambda t: (t[0], t[0].split(" "))).mapValues(lambda x: len(x))
print tweets_words.take(2)

[(u'@DisneySpain @TiniStoessel LIBERA LOGOOO \n#TINI', 5), (u'RT @amctv_es: El grupo est\xe1 separado... \xbfconseguir\xe1n escapar? \xbfllegar\xe1n a Baja? \xa1El pr\xf3ximo lunes nuevo episodio! #FearTWD\nhttps://t.co/vQW\u2026', 17)]


In [ ]:
# Union

In [79]:
tweets_hashtag = tweets_es_hashtags.map(lambda t: (unicodedata.normalize('NFKD',t["text"]).encode('ascii','ignore').lower(), \
                map(lambda h: unicodedata.normalize('NFKD',h["text"]).encode('ascii','ignore').lower(), t["entities"]["hashtags"])))\
                .flatMapValues(lambda x: x)
print tweets_hashtag.take(2)

[('@disneyspain @tinistoessel libera logooo \n#tini', 'tini'), ('rt @amctv_es: el grupo esta separado... conseguiran escapar? llegaran a baja? el proximo lunes nuevo episodio! #feartwd\nhttps://t.co/vqw...', 'feartwd')]


In [123]:
tweets_words = tweets_es_hashtags.map(lambda t: (t["text"], t["text"].split(" "))).mapValues(lambda x: len(x))
print tweets_words.take(2)




[(u'@DisneySpain @TiniStoessel LIBERA LOGOOO \n#TINI', 5), (u'RT @amctv_es: El grupo est\xe1 separado... \xbfconseguir\xe1n escapar? \xbfllegar\xe1n a Baja? \xa1El pr\xf3ximo lunes nuevo episodio! #FearTWD\nhttps://t.co/vQW\u2026', 17)]


In [64]:
#tweets_words2 = tweets_hashtag.flatMap(lambda t: (t[0], map(lambda h: h.split(" "), t[0])))
tweets_words2 = tweets_hashtag.map(lambda t: (t[0], t[0].split(" ").count()))
print tweets_words2

PythonRDD[60] at RDD at PythonRDD.scala:48


In [46]:
tweets_words3 = tweets_hashtag.map(lambda t: (t[0], map(lambda w: w.count(),t[0].split(" "))))
print tweets_words3.take(4)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 60, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/david/anaconda2/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/home/david/anaconda2/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/david/anaconda2/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/david/anaconda2/lib/python2.7/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
   